In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Connect to CRUD operations
from AnimalShelter import AnimalShelter

# Instantiate the class (no args needed)
db = AnimalShelter()

# Try to get records from MongoDB
try:
    records = db.read({})
    
    # Make sure we got a list
    if isinstance(records, list) and len(records) > 0:
        df = pd.DataFrame.from_records(records)
        if '_id' in df.columns:
            df.drop(columns=['_id'], inplace=True)
    else:
        print("✅ Connected, but no records found in the collection.")
        df = pd.DataFrame()
except Exception as e:
    print(f"❌ Failed to read from MongoDB: {e}")
    df = pd.DataFrame()

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load logo image
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App layout
app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'100px'})),
    html.Center(html.B(html.H1('Grazioso Salvare - CS-340 Dashboard Cameron DeShong'))),
    html.Hr(),

    html.Div([
        html.Label('Select Animal Outcome Type:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'All'},
                {'label': 'Adoption', 'value': 'Adoption'},
                {'label': 'Transfer', 'value': 'Transfer'},
                {'label': 'Euthanasia', 'value': 'Euthanasia'},
                {'label': 'Return to Owner', 'value': 'Return to Owner'}
            ],
            value='All',
            inline=True
        )
    ], style={'padding': 10}),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        filter_action="native",
        sort_action="native",
        row_selectable="single",
    ),

    html.Br(),
    html.Hr(),

    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'All':
        results = db.read({})
    else:
        results = db.read({"outcome_type": filter_type})
    if results:
        dff = pd.DataFrame.from_records(results)
        if '_id' in dff.columns:
            dff.drop(columns=['_id'], inplace=True)
        return dff.to_dict('records')
    else:
        return []


@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graph(viewData):
    if viewData is None or len(viewData) == 0:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    if 'breed' not in dff.columns:
        return []
    fig = px.bar(dff['breed'].value_counts().reset_index(),
                 x='index',
                 y='breed',
                 labels={'index': 'Breed', 'breed': 'Count'},
                 title='Number of Animals by Breed')
    return [dcc.Graph(figure=fig)]


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):  
    if viewData is None or index is None or len(index) == 0:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

app.run_server(debug=True, mode='inline')


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: 'NoneType' object is not iterable

